# 1. Generate Interferece Graph

# 2. Make Prediction from DL4RegAlloc

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
import pandas as pd
from sklearn.model_selection import train_test_split
from torchsummary import summary
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import pickle

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.__version__
print(device)

cuda


### Build Model

In [3]:
class DLRegAlloc(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm_1 = nn.LSTM(input_size=100, hidden_size=512, 
                              batch_first=True, bidirectional=True)
        self.lstm_2 = nn.LSTM(input_size=1024, hidden_size=256, 
                              batch_first=True,bidirectional=True)
        self.lstm_3 = nn.LSTM(input_size=512, hidden_size=128, 
                              batch_first=True,bidirectional=True)
        self.dropout = nn.Dropout(0.05)
        self.fc = nn.Linear(in_features=256, out_features=101)
        # self.softmax = nn.Softmax(dim=2)
        # self.time_distributed = TimeDistributed(self.fc, batch_first=True)
        
    def forward(self, x):
        # print("----: ", x.shape)
        x, _ = self.lstm_1(x)
        # print("lstm1 shape: ", x.shape)
        x = self.dropout(x)

        x, _ = self.lstm_2(x)
        # print("lstm2 shape: ", x.shape)
        x = self.dropout(x)

        x, _ = self.lstm_3(x)
        # print("lstm3 shape: ", x.shape)
        x = self.dropout(x)

        x = self.fc(x)
        # print("fc shape: ", x.shape)
        # x = self.softmax(x)
        # x = self.time_distributed(x)
        # print("time_distributed shape: ", x.shape)

        return x

### Load Checkpoint

In [5]:
loaded_model = DLRegAlloc().to(device)
loaded_model.load_state_dict(torch.load(f="torch_version_CROSSENTROPYLOSS_12_09_best.pth"))

<All keys matched successfully>

### Utils & Post Process Function

In [6]:
# ============================== Utils =========================================
def print_train_time(start: float, end: float, device: torch.device = None):
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time

def plot_loss_accuracy(history):
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['categorical_accuracy'])
    if ('val_categorical_accuracy' in history.history.keys()):
        plt.plot(history.history['val_categorical_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    if ('val_loss' in history.history.keys()):
        plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

def adBits(x, x2, y, seqsize=100):
   for i in range(x.shape[0]):
      #print(' for each train sample  ... ', i)
      loop = seqsize
      j = 0
      for loop in range(seqsize):     
          #print(' for each node ... ', j)
          adBits_1 = int(x[i][2*loop])
          #Get the next LONG since each node now has 2 LONGs for adj edges
          adBits_2 = int(x[i][2*loop+1])
          # Uncomment below if you want invalid nodes to not feed to NN before any valid nodes
          #if (adBits_1==0 and adBits_1==0 and loop!=seqsize-1):              
          #    y[i][j] = y[i][loop+1] 
          #    continue
          if (adBits_1):
              #print adBits
              for k in range(0,64):
                  bit = adBits_1 & 1
                  #print ( ' adBits&1 = ',  bit )
                  # Uncomment below if you want new nodes to only have adjacency to earlier nodes
                  if ( bit == 1):
                  #if ( bit == 1 and k<j):
                      x2[i][j][k] = 1
                      #print x2[i-1][j-1][k-1]
                  else:
                      x2[i][j][k] = 0
                      #print x2[i-1][j-1][k-1]
                  adBits_1 >>= 1
              x2[i][j][j] = 1 

          if (adBits_2):   
              #print adBits
              for k in range(64,100):
                  bit = adBits_2 & 1
                  #print ( ' adBits&1 = ',  bit )
                  if ( bit == 1):
                  #if ( bit == 1 and k<j):
                      x2[i][j][k] = 1
                      #print x2[i-1][j-1][k-1]
                  else:
                      x2[i][j][k] = 0
                      #print x2[i-1][j-1][k-1]
                  adBits_2 >>= 1
              x2[i][j][j] = 1
          j = j+1
   return x2, y

def updateLabelBits(x, y, seqsize=100):
    for i in range(x.shape[0]):
        #print(' for each train sample  ... ', i)
        label_dict = {}
        label_num = 1
        for j in range(seqsize):
            if (x[i][j][j] == 0):
                y[i][j] = 0
            else:
                if y[i][j] in label_dict:
                    y[i][j] = label_dict[y[i][j]]
                else:
                    label_dict[y[i][j]] = label_num
                    y[i][j] = label_num
                    label_num = label_num + 1
    return y

# ============================== Post Process ==================================



def post_process (x2_pred, predicted, seqsize=100):
    #Calculate the number of edges which will require correction
    invCols = 0
    edges = 0
    for i in range(x2_pred.shape[0]):
        for j in range(seqsize): # row
            for k in range(j): # col
                adj = x2_pred[i][j][k]
                if (adj == 1):
                    edges += 1
                    if (np.argmax(predicted[i][j]) == np.argmax(predicted[i][k])):
                        invCols += 1
                        
    print('Total No of edges ', edges)
    print('# of edges with invalid coloring ', invCols)
    print('Total percentage of edges with invalid colors ', invCols/edges)

def post_process_chromatic (x2_pred, predicted, seqsize=100):  
    colors_list_list = []
    for i in range(x2_pred.shape[0]):
        colors_list = []
        for j in range(seqsize):
            # Valid nodes will have below set to 1 so check the color 
            # assignment of those nodes only
            if (x2_pred[i][j][j] != 0):
                colors_list.append(np.argmax(predicted[i][j]))
        print('Colors list of graph ', i, ' is  \n', colors_list)
        chromatic_number = len(set(colors_list))
        print('Chromatic number of graph ', i, ' is  ', chromatic_number)
        colors_list_list.append(colors_list)
    return colors_list_list

def create_csv_rows(graph_name, colors_list_list_before_correction, 
                     colors_list_list_after_correction):
    csv_rows = []    
    for i in range(len(colors_list_list_before_correction)):
        row = [graph_name, i, len(set(colors_list_list_before_correction[i])), 
               len(set(colors_list_list_after_correction[i]))]        
        csv_rows.append(row)
    return csv_rows

def post_process_correction (x2_pred, predicted, colors_list_list, seqsize=100): 
  totInvCols = 0
  totEdges = 0

  for i in range(x2_pred.shape[0]):
      #maxcol = max(xpredicted[i])
      maxcol = max(colors_list_list[i])
      #print(maxcol)
      #mcol = maxcol[0]
      maxorigcol = maxcol
      mcolnew = maxcol
      #print('Maxcol = ',maxcol[0])
      #print(' ... FOR SAMPLE  ... ', i)
      invCols = 0
      edges = 0;
      newCol = 500

      for j in range(seqsize):
          #print(' ... ... FOR EACH NODE ... ...', j)
          for k in range(j):
              #print(' ... ... ... for each adjacency  ... ... ...', k)
              adj = x2_pred[i][j][k]
              #There is an edge
              if ( adj == 1 ):
                  edges += 1
                  if ( np.argmax(predicted[i][j]) == np.argmax(predicted[i][k]) ):                   
                      col_j = np.argmax(predicted[i][j])
                      col_k = np.argmax(predicted[i][k])
                      invCols += 1

                      #Check whether we can give one of the existing colors
                      foundfinalcol = 0
                      for  y in range(1,maxcol+1):
                          #print('Check for COLOR NO ... ', y)
                          if ( foundfinalcol == 1 ) :
                              #print('FOUND COLOR ALREADY  ... leave the loop')
                              break

                          foundcol = 0
                          #Check the adjacent nodes of j
                          #for  z in range(j):
                          for z in range(seqsize):
                              if j!=z:
                                  if  (   ((x2_pred[i][j][z] == 1) and (np.argmax(predicted[i][z]) == y))
                                      or  ((x2_pred[i][z][j] == 1) and (np.argmax(predicted[i][z]) == y))
                                      ):
                                      #print('[1] Adjacent node ... from ',j, '-->', z, 'color = ',xpredicted[i][z][0] )
                                      foundcol = 1
                                      #print('[1] Found Color ', y, ' for node ', z, 'from node ', j )
                                      break

                          #Finished checking the adjacent nodes of j
                          #Color y is not used by any of j's neighbours
                          #print('[1] Finished Checking the adjacent node of ... ',j,' ... foundcol = ',foundcol)
                          if ( foundcol == 0 ) :
                              #assign any prediction > 1
                              predicted[i][j][y] = 2
                              #print('[1] Reuse color ', y, ' for node ', j)
                              foundfinalcol = 1

                          else :
                              foundcol = 0                                                            
                              #Check the adjacent nodes of k
                              for z in range(seqsize):
                                  if k!=z:
                                      if  (   ((x2_pred[i][k][z] == 1) and (np.argmax(predicted[i][z]) == y))
                                          or  ((x2_pred[i][z][k] == 1) and (np.argmax(predicted[i][z]) == y))
                                          ):
                                          #print('[1] Adjacent node ... from ',j, '-->', z, 'color = ',xpredicted[i][z][0] )
                                          foundcol = 1
                                          #print('[1] Found Color ', y, ' for node ', z, 'from node ', j )
                                          break
                              #Color y is not used by any of k's neighbours
                              if ( foundcol == 0 ) :
                                  #assign any prediction > 1
                                  predicted[i][k][y] = 2
                                  #print('[2] Reuse color ', y, ' for node ', k )
                                  foundfinalcol = 1

                      # Could not color using an existing color
                      # Get a new color from 500 onwards OR use from the new 500 color number series
                      if ( foundfinalcol == 0 ) :
                           #newCol += 1
                           mcolnew += 1
                           #assign any prediction > 1
                           predicted[i][k][mcolnew] = 2
                           maxcol +=1
                           #print('Use new color ', mcolnew, ' for node ', k)

  return predicted



### Process Input

In [ ]:
test_csvs =["test/baidu.csv", "test/gkarate.csv"]

x_pred_list = []
y_pred_list = []
seq_size = 100

for i,csv in enumerate(test_csvs):
    print(csv, "information:")
    
    seq = pd.read_csv(csv, header=None, low_memory=False)
    columns = seq.columns.tolist()
    #get the adj edges
    adj_edge = np.array(seq[columns[1:2*seq_size+1]])
    #get the colors
    data_color = np.array(seq[columns[2*seq_size+1:3*seq_size+1]])
    X = np.zeros((adj_edge.shape[0], data_color.shape[1], seq_size))
    X, Y = adBits(adj_edge, X, data_color)
    Y = updateLabelBits(X, Y)
    Y = np.eye(101, dtype='float32')[Y]
    X = torch.tensor(X, dtype=torch.float32).to(device)
    Y = torch.tensor(Y, dtype=torch.float32).to(device)

    x_pred_list.append(X)
    y_pred_list.append(Y)

    print(x_pred_list[i].shape)
    print(y_pred_list[i].shape)  

import pickle

for i,csv in enumerate(test_csvs):
    print('\n------PREDICTING ',csv,'-------')
    loaded_model.eval()
    with torch.inference_mode():
        predicted = loaded_model(x_pred_list[i]) # torch.Size([1, 100, 101])
        predicted = torch.softmax(predicted, dim=2)
        # print("softmax后predicted shape: ", predicted.shape)
        predicted = predicted.clone().detach().cpu().numpy() # (100, 101)
        # print(predicted.shape) # (1, 100, 101)
        # print(np.argmax(predicted, axis=2)) # (1,100)

        x_pred = x_pred_list[i].clone().detach().cpu() # torch.Size([1, 100, 100])
        print('\nInvalid edges percentage before color correction ->')
        post_process(np.asarray(x_pred.tolist()), predicted)
        
        print('\nColors list and Chromatic number predicted by the model ->')
        colors_list_list_before_correction = post_process_chromatic(np.asarray(x_pred), predicted)
        print('\nApply color correction ->')
        predicted = post_process_correction(np.asarray(x_pred), predicted, colors_list_list_before_correction)
        print('\nColors list and Chromatic number following color correction ->')
        colors_list_list_after_correction = post_process_chromatic(np.asarray(x_pred), predicted)
        print('\nInvalid edges percentage after color correction ->')
        post_process(np.asarray(x_pred), predicted)
        results = create_csv_rows(csv.rsplit('/',-1)[-1], colors_list_list_before_correction, colors_list_list_after_correction)
        print(results)
        with open("results", "wb") as fp:   #Pickling
            pickle.dump(colors_list_list_after_correction, fp)


In [ ]:
import pickle
with open("results", "rb") as fp:
    color_list = pickle.load(fp)
print(color_list)

# 3. RegAlloc in LLVM